In [1]:
# notebooks/skills.ipynb

# RPGシミュレーター - スキルシステム（基本実装）
# 
# このノートブックでは、RPGシミュレーターで使用する基本的なスキルシステムを実装します。
# 
# ## 目的
# - 基本的なスキルクラスの実装
# - 単純な攻撃スキルの実装
# - キャラクターとスキルの連携

import json
import os
import random
from typing import Dict, List, Optional, Any

# characters.ipynbを読み込む（キャラクタークラスを使用するため）
%run ./characters.ipynb

# ## スキルの種類を定義する定数
SKILL_TYPE_ATTACK = "attack"     # 攻撃スキル
SKILL_TYPE_HEAL = "heal"         # 回復スキル（今後実装）

# スキル対象範囲
TARGET_SINGLE = "single"         # 単体対象
TARGET_ALL = "all"               # 全体対象

# ## 基本スキルクラス

class Skill:
    """RPGシミュレーターの基本スキルクラス"""
    
    def __init__(self, name: str, description: str, mp_cost: int, 
                 skill_type: str = SKILL_TYPE_ATTACK, target_type: str = TARGET_SINGLE,
                 power: int = 0):
        """スキルの初期化
        
        Args:
            name: スキル名
            description: スキルの説明
            mp_cost: 消費MP
            skill_type: スキルタイプ（攻撃、回復など）
            target_type: 対象タイプ（単体、全体）
            power: スキルの威力基本値
        """
        self.name = name
        self.description = description
        self.mp_cost = mp_cost
        self.skill_type = skill_type
        self.target_type = target_type
        self.power = power
    
    def can_use(self, user: Character) -> bool:
        """スキルが使用可能かどうかを判定
        
        Args:
            user: スキルを使用するキャラクター
            
        Returns:
            スキルが使用可能かどうか
        """
        # MPが足りていて、キャラクターが生存していれば使用可能
        return user.is_alive and user.current_mp >= self.mp_cost
    
    def use(self, user: Character, targets: List[Character]) -> Dict[str, Any]:
        """スキルを使用する（オーバーライド用の基本実装）
        
        Args:
            user: スキルを使用するキャラクター
            targets: スキルの対象キャラクターのリスト
            
        Returns:
            スキル使用結果の辞書（効果量、メッセージなど）
        """
        if not self.can_use(user):
            return {
                "success": False,
                "message": f"{user.name}はMPが足りないため{self.name}が使えない！"
            }
            
        # MPを消費
        user.use_mp(self.mp_cost)
        
        # 基本実装では何も起きない
        return {
            "success": True,
            "message": f"{user.name}は{self.name}を使った！しかし何も起こらなかった..."
        }

# ## 攻撃スキルクラス

class AttackSkill(Skill):
    """攻撃を行うスキルクラス"""
    
    def __init__(self, name: str, description: str, mp_cost: int, power: int,
                 target_type: str = TARGET_SINGLE):
        """攻撃スキルの初期化
        
        Args:
            name: スキル名
            description: スキルの説明
            mp_cost: 消費MP
            power: 攻撃力係数
            target_type: 対象タイプ（単体、全体）
        """
        super().__init__(name, description, mp_cost, SKILL_TYPE_ATTACK, target_type, power)
    
    def use(self, user: Character, targets: List[Character]) -> Dict[str, Any]:
        """攻撃スキルを使用
        
        Args:
            user: スキルを使用するキャラクター
            targets: 攻撃対象のキャラクターリスト
            
        Returns:
            スキル使用結果の辞書
        """
        base_result = super().use(user, targets)
        if not base_result["success"]:
            return base_result
            
        # 攻撃結果を記録
        result = {
            "success": True,
            "targets": [],
            "total_damage": 0
        }
        
        # 全体攻撃の場合のダメージ補正（全体攻撃は単体の70%の威力）
        power_multiplier = 0.7 if self.target_type == TARGET_ALL and len(targets) > 1 else 1.0
        
        # 各対象に対して攻撃処理
        for target in targets:
            if not target.is_alive:
                continue
                
            # ダメージ計算（基本攻撃力 - 防御力/2）
            base_damage = int(user.strength * self.power / 100)
            damage = max(1, int((base_damage - target.defense // 2) * power_multiplier))
            
            # ダメージのランダム変動（80%～120%）
            damage_variance = random.uniform(0.8, 1.2)
            damage = max(1, int(damage * damage_variance))
            
            # ダメージを与える
            target.take_damage(damage)
            
            # 対象ごとの結果を記録
            target_result = {
                "target": target,
                "damage": damage
            }
            
            result["targets"].append(target_result)
            result["total_damage"] += damage
        
        # 結果メッセージを生成
        if self.target_type == TARGET_ALL:
            result["message"] = f"{user.name}は{self.name}を使った！ 敵全体に合計{result['total_damage']}のダメージ！"
        else:
            target_msgs = []
            for t_result in result["targets"]:
                target_msgs.append(f"{t_result['target'].name}に{t_result['damage']}のダメージ")
            
            result["message"] = f"{user.name}は{self.name}を使った！ " + "、".join(target_msgs) + "！"
        
        return result

# ## サンプルスキル作成関数

def create_sample_skills() -> Dict[str, Skill]:
    """サンプルのスキル一覧を作成する
    
    Returns:
        スキル名をキー、Skillオブジェクトを値とする辞書
    """
    skills = {}
    
    # 単体攻撃スキル
    skills["斬りつけ"] = AttackSkill(
        name="斬りつけ",
        description="鋭い刃で敵を切りつける。基本の攻撃技。",
        mp_cost=5,
        power=120
    )
    
    # 全体攻撃スキル
    skills["全体斬り"] = AttackSkill(
        name="全体斬り",
        description="敵全体を斬る。威力は単体よりも低い。",
        mp_cost=15,
        power=90,
        target_type=TARGET_ALL
    )
    
    return skills

# スキルのテスト
def test_skills():
    """スキルの使用をテストする"""
    # サンプルキャラクターとスキルを作成
    hero = create_sample_character()
    hero.max_mp = 100
    hero.current_mp = 100
    enemy = create_sample_enemy()
    enemy2 = create_sample_enemy()
    enemy2.name = "スライムB"
    
    # スキル一覧を取得
    skills = create_sample_skills()
    
    print("==== スキルテスト ====")
    
    # 単体攻撃スキルのテスト
    print("\n[単体攻撃スキルのテスト]")
    attack_skill = skills["斬りつけ"]
    result = attack_skill.use(hero, [enemy])
    print(result["message"])
    print(f"{enemy.name}のHP: {enemy.current_hp}/{enemy.max_hp}")
    
    # 全体攻撃スキルのテスト
    print("\n[全体攻撃スキルのテスト]")
    aoe_skill = skills["全体斬り"]
    result = aoe_skill.use(hero, [enemy, enemy2])
    print(result["message"])
    print(f"{enemy.name}のHP: {enemy.current_hp}/{enemy.max_hp}")
    print(f"{enemy2.name}のHP: {enemy2.current_hp}/{enemy2.max_hp}")
    
    print("\n==== スキルテスト終了 ====")

# テスト実行
#test_skills()

勇者のステータス:
name: 勇者
level: 1
hp: 100/100
mp: 50/50
strength: 15
defense: 10
magic: 8
speed: 12
is_alive: True
job: 戦士
exp: 0
next_level_exp: 100
exp_to_next: 100

敵のステータス:
name: スライム
level: 1
hp: 50/50
mp: 20/20
strength: 8
defense: 5
magic: 3
speed: 6
is_alive: True
enemy_type: 通常
exp_reward: 15

簡易戦闘テスト:
勇者がスライムに攻撃！
11のダメージを与えた！
スライムのHP: 39/50

スライムが勇者に攻撃！
3のダメージを与えた！
勇者のHP: 97/100
==== スキルテスト ====

[単体攻撃スキルのテスト]
勇者は斬りつけを使った！ スライムに15のダメージ！
スライムのHP: 35/50

[全体攻撃スキルのテスト]
勇者は全体斬りを使った！ 敵全体に合計13のダメージ！
スライムのHP: 28/50
スライムBのHP: 44/50

==== スキルテスト終了 ====
